In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import sympy as sp

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [3]:
t = sp.Symbol('t', real=True)

q1 = sp.Function('q1')(t)
q2 = sp.Function('q2')(t)
q3 = sp.Function('q3')(t)

dq1 = sp.diff(q1, t)
dq2 = sp.diff(q2, t)
dq3 = sp.diff(q3, t)

m1, m2, m3 = sp.symbols('m1 m2 m3', positive=True)
l1, l2 = sp.symbols('l1 l2', positive=True)
a1, a2, a3 = sp.symbols('a1 a2 a3', real=True)
g = sp.symbols('g', positive=True)

I1 = sp.symbols('I1', positive=True)
I2 = sp.symbols('I2', positive=True)
I3 = sp.symbols('I3', positive=True)

In [4]:
x1 = a1 * sp.cos(q1)
x2 = l1 * sp.cos(q1) + a2 * sp.cos(q1 + q2)
x3 = l1 * sp.cos(q1) + l2 * sp.cos(q1 + q2) + a3 * sp.cos(q1 + q2 - q3)

y1 = a1 * sp.sin(q1)
y2 = l1 * sp.sin(q1) + a2 * sp.sin(q1 + q2)
y3 = l1 * sp.sin(q1) + l2 * sp.sin(q1 + q2) + a3 * sp.sin(q1 + q2 - q3)

dx1 = sp.diff(x1, t)
dx2 = sp.diff(x2, t)
dx3 = sp.diff(x3, t)

dy1 = sp.diff(y1, t)
dy2 = sp.diff(y2, t)
dy3 = sp.diff(y3, t)

In [11]:
v1_2 = sp.simplify(dx1**2 + dy1**2)
v2_2 = sp.simplify(dx2**2 + dy2**2)
v3_2 = sp.simplify(dx3**2 + dy3**2)

v1_2

a1**2*Derivative(q1(t), t)**2

In [8]:
def simplify_equation(eq):
    eq = eq.doit()  # Вычислить производные
    eq = sp.expand(eq)  # Раскрыть скобки
    eq = sp.collect(eq, [dq1, dq2, dq3, sp.Derivative(dq1, t), sp.Derivative(dq2, t), sp.Derivative(dq3, t)])
    eq = sp.trigsimp(eq)  # Упростить тригонометрию
    eq = sp.ratsimp(eq)  # Рациональное упрощение
    eq = sp.simplify(eq)  # Финальное упрощение
    return eq

In [9]:
K1 = 0.5 * (m1 * v1_2 + I1 * dq1**2)
K2 = 0.5 * (m2 * v2_2 + I2 * (dq1**2 + dq2**2))
K3 = 0.5 * (m3 * v3_2 + I3 * (dq1**2 + dq2**2 - dq3**2))
K = K1 + K2 + K3

P1 = m1 * g * y1
P2 = m2 * g * y2
P3 = m3 * g * y3
P = P1 + P2 + P3

L = K - P

equation1 = sp.diff(sp.diff(L, dq1), t) - sp.diff(L, q1)
equation2 = sp.diff(sp.diff(L, dq2), t) - sp.diff(L, q2)
equation3 = sp.diff(sp.diff(L, dq3), t) - sp.diff(L, q3)

equation1 = simplify_equation(equation1)
equation2 = sp.simplify(equation2)
equation3 = sp.simplify(equation3)

equation1

1.0*I1*Derivative(q1(t), (t, 2)) + 1.0*I2*Derivative(q1(t), (t, 2)) + 1.0*I3*Derivative(q1(t), (t, 2)) + 1.0*a1**2*m1*Derivative(q1(t), (t, 2)) + 1.0*a1*g*m1*cos(q1(t)) + 1.0*a2**2*m2*Derivative(q1(t), (t, 2)) + 1.0*a2**2*m2*Derivative(q2(t), (t, 2)) + 1.0*a2*g*m2*cos(q1(t) + q2(t)) - 2.0*a2*l1*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) - 1.0*a2*l1*m2*sin(q2(t))*Derivative(q2(t), t)**2 + 2.0*a2*l1*m2*cos(q2(t))*Derivative(q1(t), (t, 2)) + 1.0*a2*l1*m2*cos(q2(t))*Derivative(q2(t), (t, 2)) + 1.0*a3**2*m3*Derivative(q1(t), (t, 2)) + 1.0*a3**2*m3*Derivative(q2(t), (t, 2)) - 1.0*a3**2*m3*Derivative(q3(t), (t, 2)) + 1.0*a3*g*m3*cos(q1(t) + q2(t) - q3(t)) - 2.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + 2.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative(q1(t), t)*Derivative(q3(t), t) - 1.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative(q2(t), t)**2 + 2.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative(q2(t), t)*Derivative(q3(t), t) - 1.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative